# Run codes with the notation "#Run"

In [1]:
#Run
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

# Read the data to train

In [2]:
res = pd.read_csv('train_data.csv')
zipcode = pd.read_csv('citibike_zipcode.csv')
sales_price = pd.read_csv('sales_price.csv')


In [3]:
res = res.merge(zipcode, left_on='start_station_id', right_on='startstationid', how='left')
res = res.merge(sales_price, left_on='startzip', right_on='startzip', how='left').dropna()

In [4]:
res['icon_code'] = res['icon'].astype('category').cat.codes
res['holiday_code'] = res['holiday'].astype('category').cat.codes
res['weekday_code'] = res['weekday'].astype('category').cat.codes
res['startzip_code'] = res['startzip'].astype('category').cat.codes

In [5]:
icon_map = dict( zip( res['icon'], res['icon_code'] ) )
holiday_map = dict( zip( res['holiday'], res['holiday_code'] ) )
weekday_map = dict( zip( res['weekday'], res['weekday_code'] ) )
station_zipcode_map = dict( zip( res['start_station_id'], res['startzip_code'] ) )
station_salesprice_map = dict( zip( res['start_station_id'], res['startsaleprice'] ) )

In [6]:
cols = ['start_station_id','apparent_temperature', 'cloud_cover', 
        'humidity', 'icon_code' ,'precipAccumulation',
        'precip_intensity', 'temperature', 'uv_index', 'visibility', 'wind_speed',
        'holiday_code',  'weekday_code', 'startzip', 'startsaleprice']


In [7]:
y = rent_y = res['rent_freq'].dropna()
return_y = res['return_freq'].dropna()
x = res[cols].dropna()
x.dtypes

start_station_id          int64
apparent_temperature    float64
cloud_cover             float64
humidity                float64
icon_code                  int8
precipAccumulation      float64
precip_intensity        float64
temperature             float64
uv_index                float64
visibility              float64
wind_speed              float64
holiday_code               int8
weekday_code               int8
startzip                float64
startsaleprice          float64
dtype: object

## Split the data

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25)
return_x_train, return_x_test, return_y_train, return_y_test=train_test_split(x,return_y,test_size=0.25)

## Random Forest Regression

In [9]:
def RMSLE(y,ypred):
    y=np.nan_to_num(y)
    ypred=np.nan_to_num(ypred)
    calc=(ypred-y)**2
    return np.sqrt(np.mean(calc))

In [10]:
# from sklearn.model_selection import GridSearchCV
# import sklearn
# rmsle_scorer=sklearn.metrics.make_scorer(RMSLE,greater_is_better=False)

# clf_4_cs=RandomForestRegressor()
# param={'n_estimators':[200,300,400],'max_depth':[8,9,10]}
# grid_4_cs=GridSearchCV(clf_4_cs,param_grid=param,scoring=rmsle_scorer,cv=5,n_jobs=-1,verbose=2)
# grid_4_cs.fit(x_train, y_train)
# print ("Best params",grid_4_cs.best_params_)
# print ("RMSLE score for casual train %f" %(RMSLE(y_train, grid_4_cs.predict(x_train))))
# print ("RMSLE score for casual test %f" %(RMSLE(y_test, grid_4_cs.predict(x_test))))

### Rent RFR

In [ ]:
#DO NOT RERUN!!!
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score

OS = []
rfr_rent = RandomForestRegressor(n_estimators=300,max_depth=10)

rfr_rent.fit(x_train, y_train)
rent_score = rfr_rent.score(x_test, y_test)
  

In [ ]:
rent_score

### Return RFR

In [ ]:
rfr_return = RandomForestRegressor(n_estimators=300,max_depth=10)

rfr_return.fit(return_x_train, return_y_train)
return_score = rfr_return.score(return_x_test, return_y_test)


In [ ]:
return_score

In [120]:
from sklearn import preprocessing

# temp = res.iloc[:,1:]
# temp['holiday'] = temp['holiday'].astype('category').cat.codes
# x_temp = pd.get_dummies(temp)

# x_temp.head()

In [121]:
#Whitening+ extract precipaccumulation
# x_temp[['apparent_temperature',\
#        'cloud_cover', 'humidity', 'precipAccumulation', 'precip_intensity',\
#        'temperature', 'uv_index', 'visibility', 'wind_speed',]] = \
# preprocessing.scale(x_temp[['apparent_temperature',\
#        'cloud_cover', 'humidity', 'precipAccumulation', 'precip_intensity',\
#        'temperature', 'uv_index', 'visibility', 'wind_speed',]])


# x_temp.head()

In [122]:
rfr_rent.feature_importances_

array([0.23430307, 0.10380016, 0.0451671 , 0.08123278, 0.00918677,
       0.00123531, 0.01153508, 0.07808863, 0.11165856, 0.02828313,
       0.09070764, 0.00088497, 0.03613078, 0.14145386, 0.02633215])

In [123]:
rfr_return.feature_importances_

array([0.1853444 , 0.09805124, 0.04125558, 0.07767934, 0.01604533,
       0.00254128, 0.02185519, 0.08099061, 0.07631418, 0.02925463,
       0.15351587, 0.00042655, 0.03825084, 0.15053037, 0.02794459])

## Gradient Boosting on Regression Tree

In [124]:
# DO NOT RERUN!!!
#plot the accuracy
from sklearn.ensemble import GradientBoostingRegressor


OS = []

rent_clf = GradientBoostingRegressor(n_estimators=2000, alpha = 0.01) 
rent_clf.fit(x_train, y_train)
MSE=RMSLE(y_train, rent_clf.predict(x_train))
OS.append(MSE)


In [125]:
print(OS)

[0.39848039063792623]


### Return CLF

In [126]:
OS = []
# for c in N:
# c = int(c)
return_clf = GradientBoostingRegressor(n_estimators=2000, alpha = 0.01) 
return_clf.fit(return_x_train, return_y_train)
MSE=RMSLE(return_y_train, return_clf.predict(return_x_train))

## SVR

In [ ]:
#find the best
from sklearn.svm import SVR
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

rmsle_scorer=make_scorer(RMSLE,greater_is_better=False)

rg_svr=SVR()
param={'C': np.logspace(-3, 2, 2), 'gamma': np.logspace(-3, 2, 2), 'kernel' :['rbf','sigmoid']}
grid_4_cs=GridSearchCV(rg_svr,param_grid=param,scoring=rmsle_scorer,cv=5,n_jobs=-1,verbose=2)
grid_4_cs.fit(x_train, y_train)
print ("Best params",grid_4_cs.best_params_)
print ("RMSLE score for casual train %f" %(RMSLE(y_train, grid_4_cs.predict(x_train))))
print ("RMSLE score for casual test %f" %(RMSLE(y_test, grid_4_cs.predict(x_test))))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [ ]:
#plot the accuracy
C = np.logspace(-3, 2, 6)
OS = []
for c in C:
    clf = svm.SVR(kernel='sigmoid',C=c) 
    clf.fit(x_train, y_train)
    MSE= RMSLE(y_train, grid_4_cs.predict(x_train))
    OS.append(MSE)

plt.gca()
plt.plot(np.logspace(-3, 2, 6),OS)
plt.xlabel("log C")
plt.ylabel("OS accuracy")
plt.title("Accuracy vs. penalization constant (log C)")

plt.show()

In [ ]:
#need modified according to the previous result
from sklearn.svm import SVR
#need to change the kernel from Gaussian Kernel to RBM
svr = SVR(kernel='sigmoid',C=np.log(2))#
y_train_Log = np.log1p(y_train)
svr.fit(x_train,y_train_Log)
y_test_Log = np.log1p(y_test)
preds = svr.predict(X= x_test)
print ("RMSLE Value For Gradient Boost: ",RMSLE(y_test,preds,False))

In [ ]:
imshow(confusion_matrix(np.exp(y_test_Log),np.exp(preds)))

## Dock Prediction

In [31]:
from datetime import datetime
import requests
import holidays

In [65]:
def weatherdata(apikey,timestamp):
    '''get hourly data for this day 
    apikey:Darksky api 1000/day 
    timestamp: unix time'''

    r = requests.get('https://api.darksky.net/forecast/'+ \
                     apikey +'/42.3601,-71.0589,'+ "1"+ str(timestamp) + '?', auth=('user', 'pass'))
    temp = r.json()
    t = temp['hourly']['data']
    return t

In [67]:
apikey6 = 'ace6c9f9642da5c3b2a158ead33aa0d1'

r = weatherdata(apikey6, 458363600)

In [76]:


us_holidays = holidays.US()

In [42]:
x_train.dtypes

start_station_id          int64
apparent_temperature    float64
cloud_cover             float64
humidity                float64
icon_code                  int8
precipAccumulation      float64
precip_intensity        float64
temperature             float64
uv_index                float64
visibility              float64
wind_speed              float64
holiday_code               int8
weekday_code               int8
startzip                float64
startsaleprice          float64
dtype: object

In [81]:
print  datetime.fromtimestamp(1458360000).date() in us_holidays

False


In [102]:
def dockAvailPred(start_time_stampe = 1458360000, station_id = 229, init_avaiable_dock = None):
    init_dock = 10
    if init_avaiable_dock:
        init_dock = init_avaiable_dock

    dt_object = datetime.fromtimestamp(start_time_stampe)  
    res = []

    holiday = datetime.fromtimestamp(start_time_stampe).date() in us_holidays
    for hour in range(24):
        weather_data = r[hour]       
        x = {
            'start_station_id': station_id, 
            'apparent_temperature': weather_data['apparentTemperature'],
            'cloud_cover': weather_data['cloudCover'], 
            'humidity': weather_data['humidity'], 
            'icon_code':  icon_map[r[0]['icon']], 
            'precipAccumulation': weather_data['precipProbability'],
            'precip_intensity': weather_data['precipIntensity'], 
            'temperature': weather_data['temperature'], 
            'uv_index': weather_data['uvIndex'], 
            'visibility': weather_data['visibility'],
            'wind_speed': weather_data['windSpeed'], 
            'holiday_code': holiday_map[holiday], 
            'weekday_code': dt_object.weekday(), 
            'startzip': station_zipcode_map[station_id],
            'startsaleprice':station_salesprice_map[station_id]

        }
        x_pred = pd.DataFrame(x, index=[0])
        y_pred_rent = rent_clf.predict(x_pred)[0]
        y_pred_return = return_clf.predict(x_pred)[0]
        res.append({
            'rent' : y_pred_rent,
            'return': y_pred_return,
            'available': init_dock + y_pred_return - y_pred_rent
        })
        init_dock = init_dock + y_pred_return - y_pred_rent
    return res

In [103]:
pred_info = dockAvailPred()

In [104]:
print(pred_info)

[{'available': 11.854788694495669, 'return': 7.072163888233733, 'rent': 5.217375193738063}, {'available': 13.70957738899134, 'return': 7.072163888233733, 'rent': 5.217375193738063}, {'available': 15.53203892891133, 'return': 7.036162362062639, 'rent': 5.2137008221426475}, {'available': 17.413091435275206, 'return': 7.09639432466569, 'rent': 5.215341818301814}, {'available': 19.267880129770873, 'return': 7.072163888233733, 'rent': 5.217375193738063}, {'available': 21.231614519895487, 'return': 7.157014046312758, 'rent': 5.193279656188144}, {'available': 23.19905676383431, 'return': 7.141699799401918, 'rent': 5.174257555463097}, {'available': 25.05384545832998, 'return': 7.072163888233733, 'rent': 5.217375193738063}, {'available': 26.70913531189261, 'return': 7.042430461841069, 'rent': 5.387140608278444}, {'available': 28.37119564102565, 'return': 6.819362959634657, 'rent': 5.1573026305016185}, {'available': 30.382697709807157, 'return': 6.864679365477477, 'rent': 4.853177296695972}, {'a